In [1]:
import os
import numpy as np
import math
import itertools
import datetime
import time

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable


import torch.nn as nn
import torch.nn.functional as F
import torch

import glob

In [2]:
# from PIL import Image

# img = Image.open('Generate_1/trainA/9_101.jpg')
# print(img.size)#256*256

In [3]:
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

# build datast and dataloader
#train A -> train B
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned
        if mode != "train":
            self.files_A = sorted(glob.glob(os.path.join(root, "teA" ) + "/*.jpg"))
            self.files_B = sorted(glob.glob(os.path.join(root, "teB" ) + "/*.jpg"))
        else:
            self.files_A = sorted(glob.glob(os.path.join(root, "trainA" ) + "/*.jpg"))
            self.files_B = sorted(glob.glob(os.path.join(root, "trainB" ) + "/*.jpg"))

    def __getitem__(self, index):
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

            
        # resize here
#         print(image_A.size)
        image_A = image_A.resize((64,64))
        image_B = image_B.resize((64,64))
        
        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))
    

# build model    
import torch.nn as nn

import torch.nn.functional as F

import torch

# def weights_init_normal(m):
#     classname = m.__class__.__name__
#     if classname.find("Conv") != -1:
#         torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
#         if hasattr(m, "bias") and m.bias is not None:

#             torch.nn.init.constant_(m.bias.data, 0.0)

#     elif classname.find("BatchNorm2d") != -1:
#         torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
#         torch.nn.init.constant_(m.bias.data, 0.0)

class ResidualBlock(nn.Module):

    def __init__(self, in_features):

        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),#填充
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )
    def forward(self, x):
        return x + self.block(x)
#后续尝试Unet
# class GeneratorResNet(nn.Module):
#     def __init__(self, input_shape, num_residual_blocks):
#         super(GeneratorResNet, self).__init__()
#         channels = input_shape[0]
#         # Initial convolution block
#         out_features = 64
#         model = [
#             nn.ReflectionPad2d(channels),
#             nn.Conv2d(channels, out_features, 7),
#             nn.InstanceNorm2d(out_features),
#             nn.ReLU(inplace=True),
#         ]

#         in_features = out_features
#         # Downsampling
#         for _ in range(2):
#             out_features *= 2
#             model += [
#                 nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
#                 nn.InstanceNorm2d(out_features),
#                 nn.ReLU(inplace=True),
#             ]
#             in_features = out_features

#         # Residual blocks
#         for _ in range(num_residual_blocks):
#             model += [ResidualBlock(out_features)]
#         # Upsampling
#         for _ in range(2):
#             out_features //= 2
#             model += [

# #                 nn.Upsample(scale_factor=2),
#                 nn.UpsamplingNearest2d(scale_factor=2),# Updated
#                 nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
#                 nn.InstanceNorm2d(out_features),
#                 nn.ReLU(inplace=True),
#             ]
#             in_features = out_features
#      # Output layer
#         model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]
#         self.model = nn.Sequential(*model)
#     def forward(self, x):

#         return self.model(x)
def weights_init_normal(m):

    classname = m.__class__.__name__

    if classname.find("Conv") != -1:

        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)

    elif classname.find("BatchNorm2d") != -1:

        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)

        torch.nn.init.constant_(m.bias.data, 0.0)

class UNetDown(nn.Module):

    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):

        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))

        layers.append(nn.LeakyReLU(0.2))

        if dropout:

            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)
    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):

    def __init__(self, in_size, out_size, dropout=0.0):

        super(UNetUp, self).__init__()

        layers = [

            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),#反卷积

            nn.InstanceNorm2d(out_size),

            nn.ReLU(inplace=True),

        ]

        if dropout:

            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)
    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x
class GeneratorUNet(nn.Module):
    #dropout太大可能会出问题
    def __init__(self, in_channels=3, out_channels=3):

        super(GeneratorUNet, self).__init__()
        self.down1 = UNetDown(in_channels, 64, normalize=False)

        self.down2 = UNetDown(64, 128)

        self.down3 = UNetDown(128, 256)

        self.down4 = UNetDown(256, 512, dropout=0.5)

#         self.down5 = UNetDown(512, 512, dropout=0.5)

#         self.down6 = UNetDown(512, 512, dropout=0.5)

        self.down7 = UNetDown(512, 512, dropout=0.5)

        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)

#         self.up2 = UNetUp(1024, 512, dropout=0.5)

#         self.up3 = UNetUp(1024, 512, dropout=0.5)

        self.up4 = UNetUp(1024, 512, dropout=0.5)

        self.up5 = UNetUp(1024, 256)

        self.up6 = UNetUp(512, 128)

        self.up7 = UNetUp(256, 64)
        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh(),

        )
    def forward(self, x):
        #256*256
        # U-Net generator with skip connections from encoder to decoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
#         d5 = self.down5(d4)
#         d6 = self.down6(d5)
        d7 = self.down7(d4)
        d8 = self.down8(d7)
        u1 = self.up1(d8, d7)
#         u2 = self.up2(u1, d6)
#         u3 = self.up3(u2, d5)
        u4 = self.up4(u1, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)
        return self.final(u7)
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape
        # Calculate output shape of image discriminator (PatchGAN)
#         self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )
    def forward(self, img):
        return self.model(img)

In [4]:
import random
import time
import datetime
import sys
from torch.autograd import Variable
import torch
import numpy as np
from torchvision.utils import save_image


class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))


class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)


In [5]:
# process:
# stt = [
# 'parser.add_argument("--epoch", type=int, default=0, help="epoch to start training from")'
# ,'parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")'
# ,'parser.add_argument("--dataset_name", type=str, default="monet2photo", help="name of the dataset")'
# ,'parser.add_argument("--batch_size", type=int, default=1, help="size of the batches")'
# ,'parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")'
# ,'parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")'
# ,'parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")'
# ,'parser.add_argument("--decay_epoch", type=int, default=100, help="epoch from which to start lr decay")'
# ,'parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")'
# ,'parser.add_argument("--img_height", type=int, default=256, help="size of image height")'
# ,'parser.add_argument("--img_width", type=int, default=256, help="size of image width")'
# ,'parser.add_argument("--channels", type=int, default=3, help="number of image channels")'
# ,'parser.add_argument("--sample_interval", type=int, default=100, help="interval between saving generator outputs")'
# ,'parser.add_argument("--checkpoint_interval", type=int, default=-1, help="interval between saving model checkpoints")'
# ,'parser.add_argument("--n_residual_blocks", type=int, default=9, help="number of residual blocks in generator")'
# ,'parser.add_argument("--lambda_cyc", type=float, default=10.0, help="cycle loss weight")'
# ,'parser.add_argument("--lambda_id", type=float, default=5.0, help="identity loss weight")'
# ]

# for line in stt:
#     name = line[line.find('--')+2:line.find(',')-1]
#     num = line[line.find('default=')+8:line.rfind(',')]
#     print(name + '=' + num)
print(torch.__version__)
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5"
from torch.optim import lr_scheduler


1.4.0


In [6]:
#多GPU
device_ids = [0,1,2]
# parser = argparse.ArgumentParser()
# parser.add_argument("--epoch", type=int, default=0, help="epoch to start training from")
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--dataset_name", type=str, default="monet2photo", help="name of the dataset")
# parser.add_argument("--batch_size", type=int, default=1, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--decay_epoch", type=int, default=100, help="epoch from which to start lr decay")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--img_height", type=int, default=256, help="size of image height")
# parser.add_argument("--img_width", type=int, default=256, help="size of image width")
# parser.add_argument("--channels", type=int, default=3, help="number of image channels")
# parser.add_argument("--sample_interval", type=int, default=100, help="interval between saving generator outputs")
# parser.add_argument("--checkpoint_interval", type=int, default=-1, help="interval between saving model checkpoints")
# parser.add_argument("--n_residual_blocks", type=int, default=9, help="number of residual blocks in generator")
# parser.add_argument("--lambda_cyc", type=float, default=10.0, help="cycle loss weight")
# parser.add_argument("--lambda_id", type=float, default=5.0, help="identity loss weight")
# opt = parser.parse_args()
# print(opt)
#change opt to const

epoch=0
n_epochs=100
dataset_name="Generate1"
batch_size=512#改size啊...
lr=0.0002
b1=0.5
b2=0.999
decay_epoch=50
# n_cpu=2


img_height=64
img_width=64
channels=3

# sample_interval=400
checkpoint_interval=10
n_residual_blocks=9
lambda_cyc=10.0
lambda_id=5.0


# Create sample and checkpoint directories
os.makedirs("images_U/%s" % dataset_name, exist_ok=True)
os.makedirs("saved_models_U/%s" % dataset_name, exist_ok=True)

# Losses
criterion_GAN = torch.nn.MSELoss()
# criterion_GAN = torch.nn.BCELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

cuda = torch.cuda.is_available()
print('cuda avaliable:',cuda)

input_shape = ( channels,  img_height,  img_width)

# Initialize generator and discriminator
# build models
print("Building models")
# G_AB = GeneratorResNet(input_shape,  n_residual_blocks)
# G_BA = GeneratorResNet(input_shape,  n_residual_blocks)

G_AB = GeneratorUNet()
G_BA = GeneratorUNet()
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)
# 多GPU？

# model_list = [G_AB,G_BA,D_A,D_B]

if cuda:
    print('generator cudaing')
#     G_AB = G_AB.cuda()

    G_AB = torch.nn.DataParallel(G_AB, device_ids=device_ids)
    G_AB = G_AB.cuda(device=device_ids[0])
    
    G_BA = torch.nn.DataParallel(G_BA, device_ids=device_ids)
    G_BA = G_BA.cuda(device=device_ids[0])
#     G_BA = G_BA.cuda()
    print('dis cuda')
#     D_A = D_A.cuda()
#     D_B = D_B.cuda()
    D_A = torch.nn.DataParallel(D_A, device_ids=device_ids)
    D_A = D_A.cuda(device=device_ids[0])
    D_B = torch.nn.DataParallel(D_B, device_ids=device_ids)
    D_B = D_B.cuda(device=device_ids[0])

#     for model in model_list:
#         model = torch.nn.DataParallel(model, device_ids=device_ids)
#         model = model.cuda(device=device_ids[0])
    
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

#     print("cuda over")
    
print('loading weights')
if  epoch!= 0:
    # Load pretrained models
    G_AB.load_state_dict(torch.load("saved_models_U/%s/G_AB_%d.pth" % ( dataset_name,  epoch)))
    G_BA.load_state_dict(torch.load("saved_models_U/%s/G_BA_%d.pth" % ( dataset_name,  epoch)))
    D_A.load_state_dict(torch.load("saved_models_U/%s/D_A_%d.pth" % ( dataset_name,  epoch)))
    D_B.load_state_dict(torch.load("saved_models_U/%s/D_B_%d.pth" % ( dataset_name,  epoch)))
    
# 我觉得不用?
# else:
#     # Initialize weights
#     G_AB.apply(weights_init_normal)
#     G_BA.apply(weights_init_normal)
#     D_A.apply(weights_init_normal)
#     D_B.apply(weights_init_normal)

# Optimizers
print('building Opt')
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr= lr, betas=( b1,  b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr= lr, betas=( b1,  b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr= lr, betas=( b1,  b2))

from torch.optim import lr_scheduler
# Learning rate update schedulers
# lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
lr_scheduler_G = lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR( n_epochs,  epoch,  decay_epoch).step
)
lr_scheduler_D_A = lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR( n_epochs,  epoch,  decay_epoch).step
)
lr_scheduler_D_B = lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR( n_epochs,  epoch,  decay_epoch).step
)

#学到了
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Image transformations数据增强在字体中貌似不需要？
transforms_ = [
#     transforms.Resize(int( img_height * 1.12), Image.BICUBIC),
#     transforms.RandomCrop(( img_height,  img_width)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

print("building dataloader")
# Training data loader
dataloader = DataLoader(
    ImageDataset("%s" %  dataset_name, transforms_=transforms_, unaligned=True),
    batch_size= batch_size,
    shuffle=True,
#     num_workers= n_cpu,
)

# Test data loader
val_dataloader = DataLoader(
    ImageDataset("%s" %  dataset_name, transforms_=transforms_, unaligned=False, mode="test"),
    batch_size=5,
    shuffle=True,
#     num_workers=1,
)


def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs["B"].type(Tensor))
    fake_A = G_BA(real_B)
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    G_AB.train()
    G_BA.train()
    save_image(image_grid, "images_U/%s/%s.png" % ( dataset_name, batches_done), normalize=False)




cuda avaliable: True
Building models
generator cudaing
dis cuda
loading weights
building Opt
building dataloader


In [ ]:

# ----------
#  Training
# ----------

prev_time = time.time()
for epoch in range( epoch,  n_epochs):
#     if epoch%5 == 1:
#         print("train epoch %d",epoch)
    for i, batch in enumerate(dataloader):
        
        # Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))

        # Adversarial ground truths
#         valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)
#         fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)
        valid = Variable(Tensor(np.ones((real_A.size(0), *(1,4,4)))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *(1,4,4)))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # Cycle loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # Total loss
        loss_G = loss_GAN +  lambda_cyc * loss_cycle +  lambda_id * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        #  Train Discriminator A
        # -----------------------

        optimizer_D_A.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------

        optimizer_D_B.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        # Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        # Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left =  n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log学习
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
            % (
                epoch,
                 n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )

        # If at sample interval save image,是否需要截断？
#         if batches_done %  sample_interval == 0:
#             sample_images(batches_done)

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    if  checkpoint_interval != -1 and (epoch+1) %  checkpoint_interval == 0:
        # Save model checkpoints
        torch.save(G_AB.state_dict(), "saved_models_U/%s/G_AB_%d.pth" % ( dataset_name, epoch))
        torch.save(G_BA.state_dict(), "saved_models_U/%s/G_BA_%d.pth" % ( dataset_name, epoch))
        torch.save(D_A.state_dict(), "saved_models_U/%s/D_A_%d.pth" % ( dataset_name, epoch))
        torch.save(D_B.state_dict(), "saved_models_U/%s/D_B_%d.pth" % ( dataset_name, epoch))
        sample_images(epoch)

[Epoch 47/100] [Batch 74/98] [D loss: 0.180708] [G loss: 0.807830, adv: 0.316804, cycle: 0.039610, identity: 0.018986] ETA: 7:29:50.42602550571665